In [ ]:
!pip install -U pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install -U setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
!pip install autogluon==1.3.1

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 202.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import random

from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train = pd.read_parquet('train_emb_final.parquet')
test = pd.read_parquet('test_emb_final.parquet')
submission = pd.read_csv(f'../data_preprocessing/dacon/sample_submission.csv')

In [ ]:
from my_metrics import custom_score_func
from autogluon.core.metrics import make_scorer

my_metric = make_scorer(name='custom_metric', score_func=custom_score_func, optimum=1, greater_is_better=True)

In [ ]:
def clean_column_names(columns):
    return [
        f'feat_{i}' if (not name or not str(name).isidentifier()) else str(name)
        for i, name in enumerate(columns)
    ]

target_col = 'Inhibition'
y = train[target_col]
X_train = train.drop(columns=[target_col])
X_test = test.copy()

clean_cols = clean_column_names(X_train.columns)
X_train.columns = clean_cols
X_test.columns = clean_cols

In [ ]:
train_cleaned = X_train.copy()
train_cleaned[target_col] = y

train_data = TabularDataset(train_cleaned)
test_data = TabularDataset(X_test)

In [ ]:
save_path = "AG_Fin"

predictor = TabularPredictor(
    label='Inhibition',
    eval_metric=my_metric,
    problem_type='regression',
    path=save_path
).fit(
    train_data,
    excluded_model_types=["KNN", "CAT", "RF", "XT", "TABPFNMIX"],
    presets='experimental_quality',
    fit_strategy="parallel",
    time_limit=21600
)

In [ ]:
predictions = predictor.predict(test_data)
y_pred = pd.DataFrame(predictions, columns=['Inhibition'])

submission['Inhibition'] = y_pred['Inhibition']
submission.to_csv('AG_fin.csv',index=False)

# TTA

In [ ]:
test_aug = pd.read_csv('test_aug_724_emb_final.csv')
submission_t = pd.read_csv(f'../data_preprocessing/dacon/sample_submission.csv')
aug_test_tab = TabularDataset(test_aug)
test_aug['Pred'] = predictor.predict(aug_test_tab.drop(['MolID'],axis=1))
tta_predictions = test_aug.groupby('MolID')['Pred'].mean().reset_index()
tta_predictions.rename(columns={'Pred': 'Pred_TTA'}, inplace=True)
submission_t['Inhibition'] = tta_predictions['Pred_TTA']
submission_t.to_csv('submission_t.csv',index=False)

# ENS

In [ ]:
ognn_f1 = pd.read_csv('../OGNN/result/fold1_mixup_100_prediction.csv')
ognn_f2 = pd.read_csv('../OGNN/result/fold2_mixup_100_prediction.csv')
ognn_f4 = pd.read_csv('../OGNN/result/fold4_mixup_100_prediction.csv')
ognn_f5 = pd.read_csv('../OGNN/result/fold5_mixup_100_prediction.csv')

ognn = ognn_f1.copy()
ognn['Inhibition'] = (ognn_f1['Inhibition'] + ognn_f2['Inhibition'] + ognn_f4['Inhibition'] + ognn_f5['Inhibition']) / 4
sub_final = pd.read_csv(f'../data_preprocessing/dacon/sample_submission.csv')
sub_final['Inhibition'] = (submission['Inhibition'] * 0.3) + (submission_t['Inhibition'] * 0.3) + (ognn['Inhibition'] * 0.4)
sub_final.to_csv('Final.csv',index=False)